In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>

# Lime

<a id="section_intro"></a> 
## Intro


Muchas veces estamos interesados en la interpretabilidad de los modelo además de su performance.

Podemos entender la importancia de la interpretabilidad de un modelo en su posterior adopción por parte de las empresas. 

LIME (Local Interpretable Model-agnostic Explanations) es una biblioteca de Python que intenta resolver el problema de interpretabilidad de los modelos generando explicaciones a nivel local.

Para generar confianza en nuestro modelo, ejecutamos cross validation y realizamos tests sobre datos nunca antes presentados al modelo. 
Estas simulaciones nos dan una vista agregada del rendimiento del modelo sobre datos desconocidos.
Pero en general, no podemos entender por qué algunas de nuestras predicciones son correctas mientras que otras no, ni podemos rastrear el camino de decisión de nuestro modelo. En otras palabras, no podemos entender si está aprendiendo o si son conclusiones espurias.

LIME es una biblioteca de python que explica cómo decide un modelo de una manera humanamente comprensible.

LIME explica la predicción de cualquier clasificador de manera interpretable al aprender un modelo interpretable local alrededor de la predicción.

En esta guía vamos a mostrar un ejemplo de uso de LIME sobre un clasificador random forest y ver qué podemos entender de sus resultados.

**Documentación**
https://lime-ml.readthedocs.io/en/latest/lime.html

<a id="section_imports"></a> 
## Imports


In [ ]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from lime.lime_tabular import LimeTabularExplainer

#%matplotlib inline

<a id="section_dataset"></a> 
## Dataset


Para mostrar el funcionamiento de LIME vamos a usar el dataset Titanic que está disponible en https://www.kaggle.com/c/titanic-dataset/data

La variable target es `survived`, y vamos a usar como variables predictoras `pclass, sex, sibsp, parch, fare`

In [ ]:
data = pd.read_csv('../Data/titanic_train.csv')

# data preparation
data.fillna(0, inplace=True)

label_encoder = LabelEncoder()

features = ['pclass_le', 'sex_le','sibsp_le', 'parch_le', 'fare']

# label encoding textual data
data['pclass_le'] = label_encoder.fit_transform(data['pclass'])
data['sibsp_le'] = label_encoder.fit_transform(data['sibsp'])
data['sex_le'] = label_encoder.fit_transform(data['sex'])
data['parch_le'] = label_encoder.fit_transform(data['parch'])


Separamos los datos en train y test, manteniendo para test el 30% del dataset original

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data[features], data['survived'], test_size=0.3, random_state = 123)

Creamos un clasificador random forest y lo entranamos con los datos X_train, y_train

In [ ]:
random_forest_classifier = RandomForestClassifier(n_estimators=500, random_state = 123)
random_forest_classifier.fit(X_train, y_train)

Vamos a crear una instancia de `LimeTabularExplainer` 

https://lime-ml.readthedocs.io/en/latest/lime.html?highlight=limetabularexplainer#lime.lime_tabular.LimeTabularExplainer 

para después generar explicaciones sobre una predicción usando este objeto.

El constructor de `LimeTabularExplainer` recibe como parámetro `training_data` que es un numpy 2d array. Es importante pasarle los datos en un objeto de este tipo y no en un DataFrame de pandas.

Vemos de qué tipo es X_train:

In [ ]:
type(X_train)

Transformemos X_train en un numpy 2d array;

In [ ]:
X_train_explainer = np.array(X_train)
type(X_train_explainer)

Los parámetros del constructor de `LimeTabularExplainer` que vamos a usar son:
* training_data – numpy 2d array
* mode – “classification” or “regression”, <font color="green">vamos a usar classification</font>
* training_labels – labels for training data,  <font color="green">son los valores de la variable target en train</font>
* feature_names – list of names (strings) corresponding to the columns in the training data
* categorical_features – list of indices (ints) corresponding to the categorical columns. Everything else will be considered continuous. Values in these columns MUST be integers,  <font color="green">son las columnas 0 a 3 inclusive en X_train</font> 
* discretize_continuous – if True, all non-categorical features will be discretized into quartiles,  <font color="green">va a transformar la columna 4 de X_train usando deciles</font>
* discretizer – only matters if discretize_continuous is True. Options are ‘quartile’, ‘decile’ or ‘entropy’,  <font color="green">usamos deciles, pueden probar las otras variantes</font>

In [ ]:
explainer = LimeTabularExplainer(X_train_explainer, 
                                 mode = "classification",
                                 training_labels = y_train,
                                 feature_names = X_train.columns, 
                                 categorical_features  = [0, 1, 2, 3],
                                 discretize_continuous=True, 
                                 discretizer = 'decile')

Usando el objeto `explainer` y el método `explain_instance` vamos a generar explicaciones sobre una predicción concreta

https://lime-ml.readthedocs.io/en/latest/lime.html?highlight=explain_instance#lime.lime_tabular.LimeTabularExplainer.explain_instance

En este caso vamos a tomar la fila de índice 13 (puede ser cualquiera) del dataset de test, y usar `explainer` para explicar la predicción (el valor de la variable `survived`) que hace el clasificador random forest entrenado.

`explain_instance` recibe como argumentos:

* `data_row` que es de tipo 1d numpy array
* `predict_fn` – prediction function. For classifiers, this should be a function that takes a numpy array and outputs prediction probabilities. For regressors, this takes a numpy array and returns the predictions. For ScikitClassifiers, this is classifier.predict_proba(). For ScikitRegressors, this is regressor.predict(), <font color="green">usamos classifier.predict_proba()</font>
* `num_features` – maximum number of features present in explanation <font color="green">pedimos todas las variables predictoras, que son 5</font>

Y devuelve una instancia de tipo `explanation`


In [ ]:
i = 13

# necesito que sea un np array:
data_row = np.array(X_test[features].iloc[i])
explanation = explainer.explain_instance(data_row, random_forest_classifier.predict_proba, num_features=5)

La clase `explanation` provee métodos para analizar y visualizar el resultado

https://lime-ml.readthedocs.io/en/latest/lime.html?highlight=explanation#module-lime.explanation


`as_list` devuelve la lista de tuplas (feature, peso) correspondientes a la predicción

https://lime-ml.readthedocs.io/en/latest/lime.html?highlight=explanation#lime.explanation.Explanation.as_list


In [ ]:
explanation.as_list()

`as_map` devuelve un diccionario que asocia una etiqueta target a una lista de tuplas (feature_id, peso) asociadas a la predicción

https://lime-ml.readthedocs.io/en/latest/lime.html?highlight=explanation#lime.explanation.Explanation.as_map

In [ ]:
explanation.as_map()

In [ ]:
print(features[1], features[0], features[3], features[4], features[2])

`show_in_notebook` muestra en una celda de la notebook el mismo resultado que guardamos con `save_to_file`

https://lime-ml.readthedocs.io/en/latest/lime.html?highlight=explanation#lime.explanation.Explanation.show_in_notebook



In [ ]:
explanation.show_in_notebook(show_table=True)

El gráfico tiene tres partes:

* El panel de la izquierda muestra la probabilidad predicha por el modelo (random forest en este caso) para el registro de índice 13 de pertenecer a la clase 0 y de pertencer a la clase 1

* El panel del medio muestra las features por orden de importancia. Como en este caso es una clasificación binaria, vemos dos colores. Las features que tienen color naranja son compatibles con la clase 1 y los que tienen color azul son compatibles con la clase 0. 
Sex_le = 1 significa que cuando el valor de esta feature satisface este criterio, admite la clase 0. 
El número de coma flotante en las barras horizontales representa la importancia relativa de estas features.

* El panel de la derecha usa el mismo código de color que los otros dos. Contiene los valores que corresponden a la fila del DataFrame (la de índice 13) cuya predicción estamos explicando.



Veamos que el panel de la derecha efectivamente representa los datos de la fila de índice 13 del DataFrame de test

In [ ]:
X_test[features].iloc[13]

`as_pyplot_figure` devuelve un gráfico de barras que representa explanation, análogo al panel central de `show_in_notebook`

https://lime-ml.readthedocs.io/en/latest/lime.html?highlight=explanation#lime.explanation.Explanation.as_pyplot_figure



In [ ]:
explanation.as_pyplot_figure();

`save_to_file` guarda el resultado, idéntico al generado por `show_in_notebook`, en un archivo html 

https://lime-ml.readthedocs.io/en/latest/lime.html?highlight=explanation#lime.explanation.Explanation.save_to_file

(Pueden revisar que esté generado en el mismo directorio donde tienen esta notebook)

In [ ]:
explanation_path = "./explanation.html"
explanation.save_to_file(explanation_path)

## Ejercicio:

Intenten explicar las predicciones de otras instancias (filas del dataset) de test

Analicen cómo cambian las probabilidades y los pesos de las variables para distintos datos de entrada.

<a id="section_referencias"></a> 
## Referencias


Documentación
https://lime-ml.readthedocs.io/en/latest/lime.html

Decrypting your Machine Learning model using LIME 
https://towardsdatascience.com/decrypting-your-machine-learning-model-using-lime-5adc035109b5
    
Paper
https://arxiv.org/abs/1602.04938

Github
https://github.com/marcotcr/lime

Blog del autor del paper
https://homes.cs.washington.edu/~marcotcr/blog/lime/